##### __Власний обчислювальний граф__

In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.python.framework.ops import EagerTensor


def formula(x: EagerTensor, y: EagerTensor, b: EagerTensor) -> EagerTensor:
    """Some operations with tensors."""
    x = tf.matmul(x, y)
    x = x + b

    return x


function_that_uses_a_graph = tf.function(formula)  # приймає у якості аргумента звичайну функцію і генерує обчислювальний граф.
# tf.function можна використовувати як декоратор

x1 = tf.constant([[1.0, 2.0]])  # створюємо матрицю як тензор
y1 = tf.constant([[2.0], [3.0]])  # створюємо матрицю як тензор
b1 = tf.constant(4.0)  # створюємо скаляр як тензор

z1 = tf.convert_to_tensor(np.array([1, 2, 3]))  # перетворюємо масив на тензор
z1 = z1.numpy()  # зворотньо перетворюємо тензор на масив


orig_value = formula(x1, y1, b1).numpy()
tf_function_value = function_that_uses_a_graph(x1, y1, b1).numpy()

assert(orig_value == tf_function_value)

In [22]:
assert orig_value != tf_function_value, 'oops!'

AssertionError: oops!

In [23]:
# tf.function можна використовувати як декоратор
@tf.function
def function_that_uses_a_graph(x: EagerTensor, y: EagerTensor, b: EagerTensor) -> EagerTensor:
    """Some operations with tensors."""
    x = tf.matmul(x, y)
    x = x + b

    return x


tf_function_value = function_that_uses_a_graph(x1, y1, b1).numpy()

У нейронних мережах часто використовується активаційна функція ReLU:<br>
- `f(x) = max(x, 0)`

In [24]:
import tensorflow as tf


@tf.function
def relu_activation(x):
    if tf.greater(x, 0):
        return x
    
    return 0


print(relu_activation(tf.constant(1)).numpy())
print(relu_activation(tf.constant(-1)).numpy())

1
0


##### __Автоматичне диференціювання__

In [25]:
def f(x):
    return 1 / x ** 2

Тепер похідну можна обчислити так:

In [26]:
import tensorflow as tf


x = tf.Variable(2.0)
"""
За допомогою класу GradientTape tensorflow записує всі операції,
необхідні для обчислення похідної в змінну tape . Для того, щоб обчислити
значення похідної в конкретній точці, потрібно викликати метод gradient
"""
with tf.GradientTape() as tape:
    y = f(x)
    dydx = tape.gradient(y, x)
    print(dydx)

tf.Tensor(-0.25, shape=(), dtype=float32)


##### __Створення нейронної мережі__

Для того, щоб створити шар нейромережі, достатньо написати клас, який успадковується від `tf.Module`.

In [27]:
"""
Нехай потрібно написати
примітивну нейронну мережу, яка обчислює значення виразу 
w*x + b
"""
import tensorflow as tf


class SimpleModule(tf.Module):

    def __init__(self, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(5.0)
        self.b = tf.Variable(5.0)

    def __call__(self, x):
        """Обов'язковою є реалізація цтого магічного методу, 
        оскільки всі фактичні обчислення відбуваються саме тут."""
        return self.w * x + self.b
    
simple_module = SimpleModule(name='simple')
simple_module(tf.constant(5.0)).numpy()

30.0

Тепер ми можемо спробувати створити складнішу нейронну мережу.
Нехай потрібно створити нейромережу, в якій 2 шари. На першому шарі
має 3 нейрони з трьома входами, на другому - 1, в якості активаційної
функції використовуватимемо ReLU. Реалізація такої нейромережі
представлена далі.

In [28]:
class DenseLayer(tf.Module):
    """Цей клас є універсальним, оскільки за рахунок параметрів in_features та
    out_features ми можемо легко налаштовувати кількість входів та виходів
    шару.
    """
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(
                             tf.random.normal([in_features, out_features]), 
                             name='w'
                             )
        self.b = tf.Variable(tf.zeros([out_features]), name='b')
    
    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b

        return tf.nn.relu(y)


class NN(tf.Module):
    """NN (скорочення від Neural Network), який інкапсулює
    необхідні два шари і в якому визначено перетворення в магічному методі
    def __call__(self, x) . Таким чином, можна створювати нейронні
    мережі майже будь-якої складності.
    """
    def __init__(self, name=None):
        super().__init__(name=name)
        self.layer_1 = DenseLayer(in_features=3, out_features=3)
        self.layer_2 = DenseLayer(in_features=3, out_features=1)

    def __call__(self, x):
        x = self.layer_1(x)

        return self.layer_2(x)
    

nn = NN(name='neural_network')
print('Results:', nn(tf.constant([[2.0, 2.0, 2.0]])))

Results: tf.Tensor([[0.]], shape=(1, 1), dtype=float32)


##### __Навчання нейронної мережі__

маємо лінійну модель виду:<br>
a(x) = ω1 * x1 + ω2 * x2 + ... + ωn * xn + b<br>
Створимо клас для даної моделі.

In [29]:
import tensorflow as tf


class LinearModel(tf.Module):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.w = tf.Variable(5.0)
        self.b = tf.Variable(0.0)
        
    def __call__(self, x):
        return self.w * x + self.b 

Тепер визначимо дві функції: `loss` та `train`. <br>Одна буде обчислювати
помилку, а друга - підлаштовувати ваги.

In [30]:
def loss(target_y, predicted_y):
    return tf.reduce_mean(tf.square(target_y - predicted_y))

def train(model, x, y, learning_rate):
    with tf.GradientTape() as t:
        current_loss = loss(y, model(x))
         # використовуємо механізми tensorflow для автоматичного диференціювання:
        dw, db = t.gradient(current_loss, [model.w, model.b]) 
        model.w.assign_sub(learning_rate * dw) 
        # Для оновлення ваг ми використовуємо метод assign_sub , який веде себе як оператор -=
        model.b.assign_sub(learning_rate * db)

знадобиться ще одна функція, у якій ми
будемо безпосередньо тренувати модель:

In [31]:
def training_loop(model, x, y):
    for epoch in range(10):
        train(model, x, y, learning_rate=0.1)
        current_loss = loss(y, model(x))
        print(f'loss: {current_loss}')

щоб протестувати навчання моделі згенеруємо тестові дані

In [36]:
TRUE_W = 3.0
TRUE_B = 2.0
NUM_EXAMPLES = 1000
x = tf.random.normal(shape=[NUM_EXAMPLES])
noise = tf.random.normal(shape=[NUM_EXAMPLES])
y = x * TRUE_W + TRUE_B + noise

##### __Збірка та Запуск__

In [37]:
linear_model = LinearModel()
training_loop(linear_model, x, y)

loss: 6.386050224304199
loss: 4.425792217254639
loss: 3.1793289184570312
loss: 2.3867335319519043
loss: 1.882735013961792
loss: 1.562245488166809
loss: 1.3584457635879517
loss: 1.2288470268249512
loss: 1.146432638168335
loss: 1.0940228700637817


In [41]:
linear_model

In [42]:
linear_model.name

'linear_model'

In [45]:
linear_model.b.numpy()

1.8337007

In [46]:
linear_model.w.numpy()

3.170131